In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
import torch
import torch.nn as nn

In [ ]:
df=pd.read_csv('FB.csv')
open_prices=df['Open']
seqlen=15
print(open_prices)
mm=MinMaxScaler()
scaleprice=mm.fit_transform(np.array(open_prices)[...,None]).squeeze()
print(scaleprice)

In [ ]:
x=[]
y=[]
for i in range(len(scaleprice)-seqlen):
    x.append(scaleprice[i:i+seqlen])
    y.append(scaleprice[i+seqlen])
x=np.array(x)[...,None]
y=np.array(y)[...,None]


In [ ]:
train_x=torch.from_numpy(x[:int(0.8*x.shape[0])]).float()
train_y=torch.from_numpy(y[:int(0.8*x.shape[0])]).float()
test_x=torch.from_numpy(x[int(0.8*x.shape[0]):]).float()
test_y=torch.from_numpy(y[int(0.8*x.shape[0]):]).float()
print(train_x.shape,test_x.shape)

In [ ]:
class Model(nn.Module):
    def __init__(self,input_size,hidden_size):
        super().__init__()
        self.lstm=nn.LSTM(input_size,hidden_size,batch_first=True)
        self.fc=nn.Linear(hidden_size,1)
    def forward(self,x):
        output,(hidden,cell)=self.lstm(x)
        return self.fc(hidden[-1,:])
model=Model(1,64)  
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
loss_fn=nn.MSELoss()

num_epochs=100
for epoch in range(num_epochs):
    output=model(train_x)
    loss=loss_fn(output,train_y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if(epoch%10==0 and epoch!=0):
        print(epoch,"epoch loss",loss.detach().numpy())
model.eval()
with torch.no_grad():
    output=model(test_x)
pred=mm.inverse_transform(output.numpy())
real=mm.inverse_transform(test_y.numpy())

plt.plot(pred.squeeze(),color='red',label="predicted")
plt.plot(real.squeeze(),color='green',label="real")
plt.show()